# Assignment 3: Example-dependent cost-sensitive regression

*Group members:*



*   Akshay Kumar(cs23mtech11022)
*   Sanket Deone(cs23mtech11034)
*   Sanket Rathod(cs23mtech11033)
*   Abhishree Gajanan khangar(cs23mtech11021)
*   Arif Khan pathan(cs23mtech11024)

**Bahnsen approach**

In [3]:
#importing packages
import pandas as pd
import numpy as np
import math
from sklearn.utils import resample
from numpy.core.defchararray import add
from scipy.stats import norm
from joblib import Parallel, delayed, cpu_count
import itertools
from scipy.optimize import minimize
from sklearn.base import BaseEstimator
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.utils import column_or_1d


## Read dataset

In [4]:
data = pd.read_csv('/content/costsensitiveregression.csv')
data

,NotCount,YesCount,ATPM,PFD,PFG,SFD,SFG,WP,WS,AH,AN,Status,FNC
0,2,21,0.0,0.00000,0.00000,0.0,0.0000,0.000000,0.000000,0.0,0.0,0,0.000000e+00
1,23,0,0.0,0.04400,0.00000,0.0,0.0000,0.306179,0.000000,0.0,0.0,1,0.000000e+00
2,1,22,0.0,0.00000,0.00000,0.0,0.0000,0.000000,0.000000,0.0,0.0,0,0.000000e+00
3,5,18,0.0,0.00000,0.00000,0.0,0.0000,0.000000,0.000000,0.0,0.0,1,0.000000e+00
4,1,22,0.0,0.00000,0.00000,0.0,0.0000,0.000000,0.000000,0.0,0.0,0,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
147631,4,19,1.0,1.09306,4.25256,0.0,0.0000,21.946737,2.679667,0.0,0.0,0,6.092077e+05
147632,16,7,1.0,0.00000,0.00000,0.0,0.0000,0.000000,2.549239,0.0,0.0,0,6.996571e+05
147633,2,21,1.0,4.64145,5.47616,0.0,25.5783,48.283019,2.442827,0.0,0.0,0,9.658214e+05
147634,3,20,1.0,0.00000,0.40989,0.0,0.0000,0.694601,4.078739,0.0,0.0,0,1.612465e+06


In [5]:
data.describe()

,NotCount,YesCount,ATPM,PFD,PFG,SFD,SFG,WP,WS,AH,AN,Status,FNC
count,147636.000000,147636.000000,147636.000000,147636.000000,147636.000000,147636.000000,147636.000000,147636.000000,147636.000000,147636.000000,147636.000000,147636.000000,1.476360e+05
mean,7.722107,15.220908,0.253245,0.027062,0.050789,0.021648,0.071709,0.271927,0.493587,0.045045,0.013474,0.298586,5.334049e+02
std,7.574278,7.596921,0.366280,0.335509,0.373370,0.298285,1.135755,2.432425,0.763389,0.182274,0.100973,0.457639,8.774011e+03
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00
25%,2.000000,10.000000,0.001084,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.820820e-01
50%,4.000000,19.000000,0.033354,0.000000,0.000000,0.000000,0.000000,0.000000,0.097875,0.000000,0.000000,0.000000,1.183562e+01
75%,13.000000,21.000000,0.394372,0.000000,0.000000,0.000000,0.000000,0.061840,0.880929,0.000000,0.000000,1.000000,1.069840e+02
max,23.000000,22.000000,1.000000,79.837480,51.944780,61.556900,209.021870,399.606420,5.000000,1.000000,1.000000,1.000000,1.703186e+06


## Normalise data keeping False Negative cost intact

In [6]:
FNC = data['FNC']
data = (data - data.min())/(data.max() - data.min())

In [7]:
data.describe()

,NotCount,YesCount,ATPM,PFD,PFG,SFD,SFG,WP,WS,AH,AN,Status,FNC
count,147636.000000,147636.000000,147636.000000,147636.000000,147636.000000,147636.000000,147636.000000,147636.000000,147636.000000,147636.000000,147636.000000,147636.000000,1.476360e+05
mean,0.335744,0.691859,0.253245,0.000339,0.000978,0.000352,0.000343,0.000680,0.098717,0.045045,0.013474,0.298586,3.131807e-04
std,0.329316,0.345315,0.366280,0.004202,0.007188,0.004846,0.005434,0.006087,0.152678,0.182274,0.100973,0.457639,5.151529e-03
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00
25%,0.086957,0.454545,0.001084,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.656202e-07
50%,0.173913,0.863636,0.033354,0.000000,0.000000,0.000000,0.000000,0.000000,0.019575,0.000000,0.000000,0.000000,6.949105e-06
75%,0.565217,0.954545,0.394372,0.000000,0.000000,0.000000,0.000000,0.000155,0.176186,0.000000,0.000000,1.000000,6.281405e-05
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000e+00


# Prepare training and testing data

In [8]:
y = data['Status']
X = data.drop(['Status'], axis = 1)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
FNC_train = X_train['FNC']
X_train.drop(['FNC'], axis = 1, inplace = True)
#X_train=(X_train-X_train.mean())/(X_train.max())
FNC_test = X_test['FNC']
X_test.drop(['FNC'], axis = 1, inplace = True)
#X_test=(X_test-X_test.mean())/(X_test.max())
X_train = X_train.values
y_train = y_train.values
X_test = X_test.values
y_test = y_test.values
FNC_train = FNC_train.values
FNC_test = FNC_test.values

### As given in problem statement,
1. True Positive Cost is 6 for all data
2. False Positive Cost is 6 for all data
3. True Negative Cost is 0 for all data
4. Flase Negative Cost is denoted by the FNC column in the dataset

In [10]:
CTP = [6] * len(X_train) # True Positive Cost
CFP = [6] * len(X_train) # False Positive Cost
CTN = [0] * len(X_train) # True Negative Cost
CFN = FNC_train # False Negative Cost

CTP_t = [6] * len(X_test) # True Positive Cost
CFP_t = [6] * len(X_test) # False Positive Cost
CTN_t = [0] * len(X_test) # True Negative Cost
CFN_t = FNC_test # False Negative Cost

#Genetic Algorithm Code

In [11]:
#I was not able to import genetic algorithm from pyea package so i have used the source code
def _partition_estimators(n_estimators, n_jobs):
    if n_jobs == -1:
        n_jobs = min(cpu_count(), n_estimators)

    else:
        n_jobs = min(n_jobs, n_estimators)

    n_estimators_per_job = (n_estimators // n_jobs) * np.ones(n_jobs,
                                                              dtype=np.int64)
    n_estimators_per_job[:n_estimators % n_jobs] += 1
    starts = np.cumsum(n_estimators_per_job)

    return n_jobs, n_estimators_per_job.tolist(), [0] + starts.tolist()

def _fitness_function_parallel(pop, fitness_function, fargs):
    return fitness_function(pop, *fargs).tolist()


class GeneticAlgorithmOptimizer:

    def __init__(self,
                 fitness_function,
                 n_parameters,
                 iters=10,
                 type_='cont',
                 n_chromosomes=10,
                 per_mutations=0.2,
                 n_elite=2,
                 fargs=(),
                 range_=(-1, 1),
                 n_jobs=1,
                 verbose=0):

        self.n_parameters = n_parameters
        self.n_chromosomes = n_chromosomes
        self.per_mutations = per_mutations
        self.iters = iters
        self.fitness_function = fitness_function
        self.fargs = fargs
        self.n_elite = n_elite
        self.type_ = type_
        self.range_ = range_
        self.n_jobs = n_jobs
        self.verbose = verbose

    def _random(self, n):
        if self.type_ == 'binary':
            temp_ = np.random.binomial(1, 0.5, size=(n, self.n_parameters)).astype(np.int)
        else:
            temp_ = (self.range_[1] - self.range_[0]) * np.random.rand(n, self.n_parameters) + self.range_[0]
        return temp_

    def _fitness_function(self):

        n_jobs, _, starts = _partition_estimators(self.n_chromosomes, self.n_jobs)

        all_results = Parallel(n_jobs=n_jobs, verbose=self.verbose, backend='multiprocessing')(
            delayed(_fitness_function_parallel)(
                self.pop_[starts[i]:starts[i + 1]],
                self.fitness_function,
                self.fargs)
            for i in range(n_jobs))

        return np.array(list(itertools.chain.from_iterable(t for t in all_results)))

    def fit(self):

        _cols_x = ['x%d' % i for i in range(self.n_parameters)]
        self.hist_ = pd.DataFrame(index=range((self.iters + 1) * self.n_chromosomes),
                                  columns=['iter', ] + _cols_x + ['cost', 'orig', ])
        self.hist_[['orig', ]] = '-1'

        self.pop_ = self._random(self.n_chromosomes)
        self.cost_ = self._fitness_function()

        filter_iter = range(0, self.n_chromosomes)
        self.hist_.loc[filter_iter, 'iter'] = 0
        self.hist_.loc[filter_iter, 'cost'] = self.cost_
        self.hist_.loc[filter_iter, _cols_x] = self.pop_

        for i in range(self.iters):

            if self.verbose > 0:
                print('Iteration ' + str(i) + ' of ' + str(self.iters))

            orig = np.empty(self.n_chromosomes, dtype='S10')
            cost_sort = np.argsort(self.cost_)

            new_pop = np.empty_like(self.pop_)
            new_pop[0:self.n_elite] = self.pop_[cost_sort[0:self.n_elite]]
            orig[0:self.n_elite] = (cost_sort[0:self.n_elite] + i * self.n_chromosomes).astype(np.str_)

            zcost = (self.cost_ - np.average(self.cost_)) / np.std(self.cost_)
            pzcost = 1 - norm.cdf(zcost)
            pcost = np.cumsum(pzcost / sum(pzcost))

            numparents = self.n_chromosomes - self.n_elite
            rand_parents = np.random.rand(numparents, 2)
            parents = np.zeros(rand_parents.shape, dtype=np.int64)
            for parent1 in range(numparents):
                for parent2 in range(2):
                    parents[parent1, parent2] = np.searchsorted(pcost, rand_parents[parent1, parent2])

                if self.type_ == 'binary':
                    rand_match = int(np.random.rand() * self.n_parameters)
                    child = self.pop_[parents[parent1, 0]]
                    child[rand_match:] = self.pop_[parents[parent1, 1], rand_match:]
                else:
                    rand_match = np.random.rand(self.n_parameters)
                    child = self.pop_[parents[parent1, 0]] * rand_match
                    child += (1 - rand_match) * self.pop_[parents[parent1, 1]]

                new_pop[self.n_elite + parent1] = child

            orig[self.n_elite:] = [','.join(row.astype(np.str_)) for row in (parents + i * self.n_chromosomes)]

            #Mutate
            m_rand = np.random.rand(self.n_chromosomes, self.n_parameters)
            m_rand[0:self.n_elite] = 1.0
            mutations = m_rand <= self.per_mutations
            num_mutations = np.count_nonzero(mutations)

            if self.type_ == 'binary':
                new_pop[mutations] = (new_pop[mutations] == 0).astype(np.int)
            else:
                new_pop[mutations] = self._random(num_mutations)[:, 0]

            rows_mutations = np.any(mutations, axis=1)
            orig[rows_mutations] = add(orig[rows_mutations],
                                       np.array(['_M'] * np.count_nonzero(rows_mutations), dtype='S10'))

            # Replace replicates with random
            temp_unique = np.ascontiguousarray(new_pop).view(np.dtype((np.void,
                                                                       new_pop.dtype.itemsize * new_pop.shape[1])))
            _, temp_unique_idx = np.unique(temp_unique, return_index=True)
            n_replace = self.n_chromosomes - temp_unique_idx.shape[0]
            if n_replace > 0:
                temp_unique_replace = np.ones(self.n_chromosomes, dtype=bool)
                temp_unique_replace[:] = True
                temp_unique_replace[temp_unique_idx] = False
                new_pop[temp_unique_replace] = self._random(n_replace)
                orig[temp_unique_replace] = '-1'

            self.pop_ = new_pop
            self.cost_ = self._fitness_function()

            filter_iter = range((i + 1) * self.n_chromosomes, (i + 2) * self.n_chromosomes)
            self.hist_.loc[filter_iter, 'iter'] = i + 1
            self.hist_.loc[filter_iter, 'cost'] = self.cost_
            self.hist_.loc[filter_iter, _cols_x] = self.pop_
            self.hist_.loc[filter_iter, 'orig'] = orig

        best = np.argmin(self.cost_)
        self.x = self.pop_[best]
        self.x_cost = self.cost_[best]

#Cost sensitive function

In [12]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [13]:
def loss_function(y_true, y_pred, cost_mat):
    y_true = column_or_1d(y_true)
    y_true = (y_true == 1).astype(np.float64)
    y_pred = column_or_1d(y_pred)
    y_pred = (y_pred == 1).astype(np.float64)
    cost = y_true * ((1 - y_pred) * cost_mat[:, 1] + y_pred * cost_mat[:, 2])
    cost += (1 - y_true) * (y_pred * cost_mat[:, 0] + (1 - y_pred) * cost_mat[:, 3])
    return np.sum(cost)


In [14]:
def c_loss(w, X, y, cost_mat, alpha):
    n_samples = X.shape[0]
    c = 0.
    if w.size == X.shape[1] + 1:
        c = w[-1]
        w = w[:-1]
    z = np.dot(X, w) + c
    y_prob = sigmoid(z)
    out = loss_function(y, y_prob, cost_mat) / n_samples
    out += .5 * alpha * np.dot(w, w)
    return out

In [15]:
def logistic_cost_loss(w, X, y, cost_mat, alpha):
    if w.shape[0] == w.size:
        return c_loss(w, X, y, cost_mat, alpha)
    else:
        n_w = w.shape[0]
        out = np.zeros(n_w)
        for i in range(n_w):
            out[i] = c_loss(w[i], X, y, cost_mat, alpha)
        return out

In [16]:
#Cost Sensitive Logistic Regression Algorithm
class CostSensitive(BaseEstimator):
    def __init__(self,C=1.0, max_iter=100,tol=1e-4,verbose=0):
        self.C = C
        self.max_iter = max_iter
        self.tol = tol
        self.coef_ = None
        self.intercept_ = 0.
        self.verbose = verbose

    def proba(self, X):
        y_prob = np.zeros((X.shape[0], 2))
        y_prob[:, 1] = sigmoid(np.dot(X, self.coef_) + self.intercept_)
        y_prob[:, 0] = 1 - y_prob[:, 1]
        return y_prob

    def fit(self, X, y, cost_mat):
        n_features = X.shape[1]
        w0 = np.zeros(n_features + 1)
        res = GeneticAlgorithmOptimizer(logistic_cost_loss, w0.shape[0],iters=self.max_iter,type_='cont',n_chromosomes=100, per_mutations=0.25,
                                            n_elite=10,fargs=(X, y, cost_mat, 1. / self.C),range_=(-5, 5), n_jobs=1, verbose=self.verbose)
        res.fit()
        self.coef_ = res.x[:-1]
        self.intercept_ = res.x[-1]

    def predict(self, X, cut_point=0.5):
        return np.floor(self.proba(X)[:, 1] + (1 - cut_point))

In [17]:
cost_matrix = np.vstack((CTP, CFP, CTN, CFN)).T
cost_matrix_test = np.vstack((CTP_t, CFP_t, CTN_t, CFN_t)).T

In [31]:
bahsen = CostSensitive()

In [32]:
bahsen.fit(X_train,y_train,cost_matrix)

In [33]:
pred = bahsen.predict(X_test)

In [34]:
print(loss_function(y_test,pred,cost_matrix_test))

52644.19800475649


In [35]:
#Calculating accuracy and misclassification
ans = pred
total_size = len(y_test)
accuracy = 0
cost_cslr = 0
for i in range(total_size):
    if y_test[i] == ans[i] and y_test[i] == 1:
        accuracy = accuracy + 1
        cost_cslr = cost_cslr + 6
    elif y_test[i] == 1 and ans[i] == 0:
        cost_cslr = cost_cslr + FNC_test[i]
    elif y_test[i] == 0 and ans[i] == 1:
        cost_cslr = cost_cslr + 6
    else:
        accuracy = accuracy + 1
accuracy = accuracy/total_size
print('Accuracy of Cost-Sensitive Logistic Regression Model: ', accuracy)
print('Misclassification Score of Cost-Sensitive Logistic Regression Model: ', cost_cslr)

Accuracy of Cost-Sensitive Logistic Regression Model:  0.7028921701435925
Misclassification Score of Cost-Sensitive Logistic Regression Model:  2.7303632209142377


In [36]:
#Training simple logistic regression and Calculating accuracy and misclassification
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression().fit(X_train, y_train)
ans = clf.predict(X_test)
cost_lr = 0
for i in range(len(X_test)):
    if y_test[i] == ans[i] and y_test[i] == 1:
        cost_lr = cost_lr + 6
    elif y_train[i] == 1 and ans[i] == 0:
        cost_lr = cost_lr + FNC_test[i]
    elif y_train[i] == 0 and ans[i] == 1:
        cost_lr = cost_lr + 6
print('Accuracy of Standard Logistic Regression Model: ', clf.score(X_test, y_test))
print('Misclassification Score of Cost-Sensitive Logistic Regression Model: ', cost_lr)

Accuracy of Standard Logistic Regression Model:  0.8664657274451368
Misclassification Score of Cost-Sensitive Logistic Regression Model:  44479.346135961554


In [37]:
print('Savings cost: ', 1 - cost_cslr/cost_lr)

Savings cost:  0.9999386150324114


#Nikou Gunnemann's approach

In [25]:
def logistic_loss(y_true, y_pred, FNC):
    epsilon = 1e-15
    loss = np.mean((FNC * y_true * -np.log(y_pred + epsilon) + FNC * (1 - y_true) * -np.log(1 - y_pred + epsilon)))
    return loss

In [26]:
def predict(X, weights):
    X = np.hstack((np.ones((X.shape[0], 1)), X))
    prob = sigmoid(np.dot(X, weights))
    preds = (prob >= 0.5).astype(int)
    return preds

In [27]:
def logistic_regression_fit(X, y, FNC, learning_rate=0.01, epochs=100):
    X = np.hstack((np.ones((X.shape[0], 1)), X))
    weight = np.random.randn(X.shape[1])

    for epoch in range(epochs):
        y_pred = sigmoid(np.dot(X, weight))
        grad = np.dot(X.T, (y_pred - y))
        weight -= learning_rate * grad
        loss = logistic_loss(y, y_pred, FNC)
        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss}")

    return weight

In [28]:
trained_weights = logistic_regression_fit(X_train, y_train, FNC_train)

Epoch 1/100, Loss: 0.00026900683109681807
Epoch 2/100, Loss: 0.0016485378885888978
Epoch 3/100, Loss: 0.004518292205086082
Epoch 4/100, Loss: 0.0031543137369427765
Epoch 5/100, Loss: 0.0012560181740986487
Epoch 6/100, Loss: 0.0013255637605049896
Epoch 7/100, Loss: 0.0013891490728494276
Epoch 8/100, Loss: 0.0013802309220229969
Epoch 9/100, Loss: 0.001361954815982423
Epoch 10/100, Loss: 0.0013537459349571212


<ipython-input-12-dc64dee5606b>:2: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))


Epoch 11/100, Loss: 0.0013168292260799638
Epoch 12/100, Loss: 0.0013139754034164118
Epoch 13/100, Loss: 0.001258874544351209
Epoch 14/100, Loss: 0.0012665773194037594
Epoch 15/100, Loss: 0.0011465437187075695
Epoch 16/100, Loss: 0.0012195182101762641
Epoch 17/100, Loss: 0.0010212384618454213
Epoch 18/100, Loss: 0.007755919522693096
Epoch 19/100, Loss: 0.0031543137369427765
Epoch 20/100, Loss: 0.0031543137369427765
Epoch 21/100, Loss: 0.0011878008734778804
Epoch 22/100, Loss: 0.001319120872831521
Epoch 23/100, Loss: 0.0013583316207578106
Epoch 24/100, Loss: 0.0013569476505384506
Epoch 25/100, Loss: 0.0013415956087375475
Epoch 26/100, Loss: 0.0013290721247600989
Epoch 27/100, Loss: 0.0013147974869354457
Epoch 28/100, Loss: 0.0012996131928908739
Epoch 29/100, Loss: 0.0012833324222430297
Epoch 30/100, Loss: 0.0012653134823729728
Epoch 31/100, Loss: 0.0012448070063189281
Epoch 32/100, Loss: 0.0012054266783202152
Epoch 33/100, Loss: 0.001139577959308505
Epoch 34/100, Loss: 0.0010582752497927

In [29]:
# Make predictions on the test set
predictions = predict(X_test, trained_weights)

In [30]:
def accuracy(y_true, y_pred):
    correct = np.sum(y_true == y_pred)
    total = len(y_true)
    accuracy = correct / total
    return accuracy

# Calculate accuracy
acc = accuracy(y_test, predictions)
print("Accuracy:", acc)


Accuracy: 0.863654836087781


#We can see that accruracy with gunnemann's approach is better than normal logistic regression